## Deliverable 2. Create a Customer Travel Destinations Map.

In [59]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [60]:
# 1. Import the WeatherPy_database.csv file.

city_data_df = pd.read_csv("../weather-database/WeatherPy_database.csv")

city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,44.11,76,75,19.57,broken clouds
1,1,Butaritari,KI,3.0707,172.7902,81.46,79,88,13.35,light rain
2,2,Narsaq,GL,60.9167,-46.0500,43.02,79,65,2.15,broken clouds
3,3,Lalmohan,BD,22.3384,90.7419,81.37,81,72,8.99,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,82.45,74,20,5.75,few clouds


In [65]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [66]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,81.46,79,88,13.35,light rain
3,3,Lalmohan,BD,22.3384,90.7419,81.37,81,72,8.99,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,82.45,74,20,5.75,few clouds
5,5,Atuona,PF,-9.8000,-139.0333,78.31,81,100,12.73,overcast clouds
7,7,Bonoua,CI,5.2741,-3.5950,83.82,89,78,4.12,light rain
8,8,Fare,PF,-16.7000,-151.0167,80.96,72,7,9.24,clear sky
16,16,Kapaa,US,22.0752,-159.3190,82.38,76,40,18.41,scattered clouds
21,21,Hilo,US,19.7297,-155.0900,81.28,82,75,12.66,broken clouds
23,23,Rikitea,PF,-23.1203,-134.9692,76.46,70,20,5.48,few clouds
25,25,Mogadishu,SO,2.0371,45.3438,82.40,78,100,14.58,moderate rain


In [67]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [68]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_clean_df = preferred_cities_df.dropna()

preferred_cities_clean_df.count()

City_ID                189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [69]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.46,light rain,3.0707,172.7902,
3,Lalmohan,BD,81.37,broken clouds,22.3384,90.7419,
4,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
5,Atuona,PF,78.31,overcast clouds,-9.8000,-139.0333,
7,Bonoua,CI,83.82,light rain,5.2741,-3.5950,
8,Fare,PF,80.96,clear sky,-16.7000,-151.0167,
16,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
21,Hilo,US,81.28,broken clouds,19.7297,-155.0900,
23,Rikitea,PF,76.46,few clouds,-23.1203,-134.9692,
25,Mogadishu,SO,82.40,moderate rain,2.0371,45.3438,


In [70]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    
    "radius": 5000,
    
    "type": "lodging",
    
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [95]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")

hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

clean_hotel_df = hotel_df

clean_hotel_df.count()

City                   176
Country                176
Max Temp               176
Current Description    176
Lat                    176
Lng                    176
Hotel Name             176
dtype: int64

In [97]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.46,light rain,3.0707,172.7902,Isles Sunset Lodge
3,Lalmohan,BD,81.37,broken clouds,22.3384,90.7419,Arifur Rahman Miraj
4,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,Paradise Inn
5,Atuona,PF,78.31,overcast clouds,-9.8000,-139.0333,Villa Enata
7,Bonoua,CI,83.82,light rain,5.2741,-3.5950,Hotel Bon Samaritian
8,Fare,PF,80.96,clear sky,-16.7000,-151.0167,Maitai Lapita Village Huahine
16,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
21,Hilo,US,81.28,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Rikitea,PF,76.46,few clouds,-23.1203,-134.9692,People ThankYou
25,Mogadishu,SO,82.40,moderate rain,2.0371,45.3438,Hotel Juba


In [98]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [99]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]

In [100]:
# 11a. Add a marker layer for each city to the map.

locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))